In [1]:
import json
import numpy as np
from termcolor import colored

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout

In [2]:
model = Sequential()
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_1'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2, name='conv_1_maxpool'))
model.add(Activation('relu', name='conv_1_act'))
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_2'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2, name='conv_2_maxpool'))
model.add(Activation('relu', name='conv_2_act'))

model.add(Flatten())
model.add(Dense(units=32, name='dense_1'))
model.add(Activation('relu', name='dense_1_act'))
model.add(Dense(units=10, activation='softmax', name='dense_2'))

model.build(input_shape=(None, 28, 28, 1)) # model build
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
conv_1_maxpool (MaxPooling2D (None, 13, 13, 10)        0         
_________________________________________________________________
conv_1_act (Activation)      (None, 13, 13, 10)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 11, 11, 10)        910       
_________________________________________________________________
conv_2_maxpool (MaxPooling2D (None, 5, 5, 10)          0         
_________________________________________________________________
conv_2_act (Activation)      (None, 5, 5, 10)          0         
_________________________________________________________________
flatten (Flatten)            (None, 250)               0

In [3]:
print(colored('model.layers', 'cyan'), '\n', model.layers, '\n')
print(colored('len(model.layers)', 'cyan'), '\n', len(model.layers), '\n')

model.layers 
 [<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002B3D63BB0D0>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002B3D63C4D60>, <tensorflow.python.keras.layers.core.Activation object at 0x000002B3D63BB0A0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002B3D63E96D0>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002B3D63E9BE0>, <tensorflow.python.keras.layers.core.Activation object at 0x000002B3D63C4D90>, <tensorflow.python.keras.layers.core.Flatten object at 0x000002B3D63E9FD0>, <tensorflow.python.keras.layers.core.Dense object at 0x000002B3D63E9F10>, <tensorflow.python.keras.layers.core.Activation object at 0x000002B3DC87A550>, <tensorflow.python.keras.layers.core.Dense object at 0x000002B3DC87A670>] 

len(model.layers) 
 10 



In [4]:
final_layer = model.layers[0] # 첫번째 conv layer
print(final_layer, '\n')
print(dir(final_layer), '\n')


['_TF_MODULE_IGNORED_PROPERTIES', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_activity_regularizer', '_add_trackable', '_add_variable_with_custom_getter', '_auto_track_sub_layers', '_autocast', '_autographed_call', '_build_input_shape', '_call_accepts_kwargs', '_call_arg_was_passed', '_call_fn_arg_defaults', '_call_fn_arg_positions', '_call_fn_args', '_call_full_argspec', '_callable_losses', '_cast_single_input', '_channels_first', '_checkpoint_dependencies', '_clear_losses', '_compute_causal_padding', '_compute_dtype', '_compute_dtype_object', '_convolution_op', '_dedup_weights', '_default_training_arg', '_deferred_dependencies', '_dtype', '_dt

In [5]:
final_layer_config = final_layer.get_config() # layer의 정보를 얻어낼 때 사용하면 유용
print(json.dumps(final_layer_config, indent=2), '\n')

{
  "name": "conv_1",
  "trainable": true,
  "dtype": "float32",
  "filters": 10,
  "kernel_size": [
    3,
    3
  ],
  "strides": [
    1,
    1
  ],
  "padding": "valid",
  "data_format": "channels_last",
  "dilation_rate": [
    1,
    1
  ],
  "groups": 1,
  "activation": "linear",
  "use_bias": true,
  "kernel_initializer": {
    "class_name": "GlorotUniform",
    "config": {
      "seed": null
    }
  },
  "bias_initializer": {
    "class_name": "Zeros",
    "config": {}
  },
  "kernel_regularizer": null,
  "bias_regularizer": null,
  "activity_regularizer": null,
  "kernel_constraint": null,
  "bias_constraint": null
} 



In [6]:
print(colored('type(final_layer_config)', 'cyan'), '\n', type(final_layer_config),'\n')
print(colored('final_layer_config.keys()', 'cyan'), '\n', final_layer_config.keys(),'\n')
print(colored('final_layer_config.values()', 'cyan'), '\n', final_layer_config.values(),'\n')

type(final_layer_config) 
 <class 'dict'> 

final_layer_config.keys() 
 dict_keys(['name', 'trainable', 'dtype', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'groups', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint']) 

final_layer_config.values() 
 dict_values(['conv_1', True, 'float32', 10, (3, 3), (1, 1), 'valid', 'channels_last', (1, 1), 1, 'linear', True, {'class_name': 'GlorotUniform', 'config': {'seed': None}}, {'class_name': 'Zeros', 'config': {}}, None, None, None, None, None]) 



In [7]:
for layer in model.layers:
  layer_config = layer.get_config()
  layer_name = layer_config['name']
  
  if layer_name.startswith('conv') and len(layer_name.split('_')) <= 2:
    print(colored('Layer name', 'cyan'), layer_name)
    print('n_filters :', layer_config['filters'])
    print('kernal_size :', layer_config['kernel_size'])
    print('padding :', layer_config['padding'])
    print()
    
  if layer_name.endswith('act'):
    print(colored('Layer name', 'cyan'), layer_name)
    print('activation :', layer_config['activation'])
    print()

Layer name conv_1
n_filters : 10
kernal_size : (3, 3)
padding : valid

Layer name conv_1_act
activation : relu

Layer name conv_2
n_filters : 10
kernal_size : (3, 3)
padding : valid

Layer name conv_2_act
activation : relu

Layer name dense_1_act
activation : relu



In [8]:
final_layer = model.layers[-1]

print(type(final_layer.get_weights()))
print(type(final_layer.get_weights()[0]))
print(type(final_layer.get_weights()[1]))
print(final_layer.get_weights()[0].shape) # weights
print(final_layer.get_weights()[1].shape) # bias

<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(32, 10)
(10,)


In [9]:
print('trainable params',
      np.prod(final_layer.get_weights()[0].shape) + np.sum(final_layer.get_weights()[1].shape))

trainable params 330
